In [1]:
import sys
!{sys.executable} -m pip install kfp >/dev/null

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
import json
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

In [3]:
dkube_preprocessing_op      = components.load_component_from_file("/mnt/dkube/pipeline/components/preprocess/component.yaml")
dkube_training_op           = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op            = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")

In [4]:
image = "docker.io/ocdr/d3-datascience-sklearn:v0.23.2"
serving_image = "ocdr/sklearnserver:0.23.2"
dataset = 'insurance'
featureset = 'insurance-fs'
training_program = 'insurance'
model = 'insurance'
preprocessing_script = f"python preprocessing.py --fs {featureset}"
training_script = f"python training.py --fs {featureset}"
transformer_code='insurance/transformer.py'
user = os.getenv('USERNAME')
framework = "sklearn"
f_version = "0.23.2"
input_mount_point = "/opt/dkube/in"
output_mount_point = "/opt/dkube/out"

In [5]:
@kfp.dsl.pipeline(
    name='dkube-insurance-pl',
    description='sample insurance pipeline with featuresets'
)
def insurance_pipeline(token):
    
    preprocessing = dkube_preprocessing_op(token, json.dumps({"image": image}),
                                           program=training_program, run_script=preprocessing_script,
                                           datasets=json.dumps([dataset]), 
                                           output_featuresets=json.dumps([str(featureset)]),
                                           input_dataset_mounts=json.dumps([input_mount_point]), 
                                           output_featureset_mounts=json.dumps([output_mount_point])
                                            )

    train       = dkube_training_op(token, json.dumps({"image": image}),
                                    framework=framework, version=f_version,
                                    program=training_program, run_script=training_script,
                                    featuresets=json.dumps([featureset]), outputs=json.dumps([model]),
                                    input_featureset_mounts=json.dumps([input_mount_point]),
                                    output_mounts=json.dumps([output_mount_point])).after(preprocessing)

    serving     = dkube_serving_op(token, train.outputs['artifact'], device='cpu', 
                                    serving_image=json.dumps({"image": serving_image}),
                                    transformer_image=json.dumps({"image": image}),
                                    transformer_project=training_program,
                                    transformer_code=transformer_code).after(train)

# Create a run

In [6]:
existing_token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(existing_token=existing_token)
client.create_run_from_pipeline_func(insurance_pipeline, arguments={"token":existing_token})

RunPipelineResult(run_id=b773fe5a-eb69-4abc-9316-f48e9118d0ce)

# Compile and generate tar ball

In [7]:
compiler.Compiler().compile(insurance_pipeline, 'dkube_insurance_fs.tar.gz')

# Upload pipeline

In [8]:
client = kfp.Client(existing_token=existing_token)
try:
    client.upload_pipeline(pipeline_package_path = 'dkube_insurance_fs.tar.gz', pipeline_name = 'Insurance pipeline', description = None)
except BaseException as e:
    print(e)